# Лабораторная работа 5. Разработка единого шаблона предварительной обработки данных

## Подключение библиотек

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Загрузка данных и разделение на матрицу признаков и зависимую переменную

In [23]:
dataset = pd.read_csv('car_accident.csv')
dataset.head()

,Gender,Expirience years,Age,Has car accident this year
0,male,1.0,21.0,Yes
1,male,13.0,38.0,No
2,female,4.0,23.0,Yes
3,male,NaN,39.0,No
4,female,8.0,44.0,No


In [24]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 3].values
print ("Матрица признаков"); print(X)
print ("Зависимая переменная"); print(y)

Матрица признаков
[['male' 1.0 21.0]
 ['male' 13.0 38.0]
 ['female' 4.0 23.0]
 ['male' nan 39.0]
 ['female' 8.0 44.0]
 ['female' 3.0 32.0]
 ['male' 12.0 nan]
 ['female' 2.0 34.0]
 ['female' 25.0 50.0]
 ['male' 24.0 61.0]]
Зависимая переменная
['Yes' 'No' 'Yes' 'No' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes']


### 3. Обработка пропущенных значений

In [9]:
# устаревший подход
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer = imputer.fit(X[:, 1:3])
X_without_nan = X.copy()
X_without_nan[:, 1:3] = imputer.transform(X[:, 1:3])
print(X_without_nan)

ImportError: cannot import name 'Imputer' from 'sklearn.preprocessing' (D:\anaconda3\lib\site-packages\sklearn\preprocessing\__init__.py)

In [25]:
# Новая версия класса-трансформера, предыдущая Imputer - устарела
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X[:, 1:3])
X_without_nan = X.copy()
X_without_nan[:, 1:3] = imputer.transform(X[:, 1:4])
X_without_nan

array([['male', 1.0, 21.0],
       ['male', 13.0, 38.0],
       ['female', 4.0, 23.0],
       ['male', 10.222222222222221, 39.0],
       ['female', 8.0, 44.0],
       ['female', 3.0, 32.0],
       ['male', 12.0, 38.0],
       ['female', 2.0, 34.0],
       ['female', 25.0, 50.0],
       ['male', 24.0, 61.0]], dtype=object)

### 4. Обработка категориальных данных

#### 4.1 Замена категории кодом (LabelEncoder)

In [26]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
print("Зависимая переменная до обработки")
print(y)
y = labelencoder_y.fit_transform(y)
print("Зависимая переменная после обработки")
print(y)

Зависимая переменная до обработки
['Yes' 'No' 'Yes' 'No' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes']
Зависимая переменная после обработки
[1 0 1 0 0 0 1 1 0 1]


#### 4.2 Применение OneHotEncoder

In [28]:
# устаревший подход к использованию OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
labelencoder_X = LabelEncoder()
labelencoder_X.fit_transform(X[:, 0])
X_encoded = X_without_nan.copy()
X_encoded[:, 0] = labelencoder_X.fit_transform(X_encoded[:, 0])
onehotencoder = OneHotEncoder()
X_encoded = onehotencoder.fit_transform(X_encoded).toarray()
print("Перекодировка категориального признака")
print(X_encoded)

Перекодировка категориального признака
[[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [29]:
# создаем копию "грязного" объекта: спропусками и некодированными категориями
X_dirty = X.copy()
X_dirty

array([['male', 1.0, 21.0],
       ['male', 13.0, 38.0],
       ['female', 4.0, 23.0],
       ['male', nan, 39.0],
       ['female', 8.0, 44.0],
       ['female', 3.0, 32.0],
       ['male', 12.0, nan],
       ['female', 2.0, 34.0],
       ['female', 25.0, 50.0],
       ['male', 24.0, 61.0]], dtype=object)

In [30]:
# Современный метод трансформации признаков
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# создаем список трансформеров
transformers = [
    ('onehot', OneHotEncoder(), [0]),
    ('imp', SimpleImputer(), [1, 2])
]

# Создаем объект ColumnTransformer и передаем ему список трансформеров
ct = ColumnTransformer(transformers)

# Выполняем трансформацию признаков
X_transformed = ct.fit_transform(X_dirty)
print(X_transformed.shape)
X_transformed

(10, 4)


array([[ 0.        ,  1.        ,  1.        , 21.        ],
       [ 0.        ,  1.        , 13.        , 38.        ],
       [ 1.        ,  0.        ,  4.        , 23.        ],
       [ 0.        ,  1.        , 10.22222222, 39.        ],
       [ 1.        ,  0.        ,  8.        , 44.        ],
       [ 1.        ,  0.        ,  3.        , 32.        ],
       [ 0.        ,  1.        , 12.        , 38.        ],
       [ 1.        ,  0.        ,  2.        , 34.        ],
       [ 1.        ,  0.        , 25.        , 50.        ],
       [ 0.        ,  1.        , 24.        , 61.        ]])

In [34]:
# Можно преобразовать полученный многомерный массив обратно в Dataframe
X_data = pd.DataFrame(
    X_transformed, 
    columns=['C1', 'C2', 'Experience years', 'Age'])
X_data

,C1,C2,Experience years,Age
0,0.0,1.0,1.000000,21.0
1,0.0,1.0,13.000000,38.0
2,1.0,0.0,4.000000,23.0
3,0.0,1.0,10.222222,39.0
4,1.0,0.0,8.000000,44.0
5,1.0,0.0,3.000000,32.0
6,0.0,1.0,12.000000,38.0
7,1.0,0.0,2.000000,34.0
8,1.0,0.0,25.000000,50.0
9,0.0,1.0,24.000000,61.0
